In [16]:
import numpy as np
import torch
from src.utils.myOptimization import (
    AlphaFairness, AlphaFairnesstorch,
    solveIndProblem, solve_closed_form,
    solve_group, compute_coupled_group_obj,
    solve_group_grad, compute_gradient_closed_form,
    compute_group_gradient_analytical, solve_d_and_gradient_analytical
)


def solve_d_and_gradient_analytical(g, r, c, alpha, Q):
    """
    Computes the optimal decision d* and its analytical gradient w.r.t. r.

    Args:
        g (np.ndarray): Gain factors, shape (n,).
        r (np.ndarray): Predicted risk values, shape (n,).
        c (np.ndarray): Cost values, shape (n,).
        alpha (float or str): Fairness parameter.
        Q (float): Total budget.

    Returns:
        tuple[np.ndarray, np.ndarray]:
            - d_star (np.ndarray): The optimal decision vector, shape (n,).
            - grad_d_r (np.ndarray): The Jacobian matrix (gradient) of d* w.r.t. r,
                                     shape (n, n), where grad[i, k] = ∂d_i*/∂r_k.
    """
    n = len(r)
    g, r, c = map(np.atleast_1d, [g, r, c])
    utility = g * r
    grad_d_r = np.zeros((n, n))

    # --- 1. Solve for d* ---
    if alpha == 0:
        # Utilitarian: non-differentiable, handled below.
        i_star = np.argmax(utility / c)
        d_star = np.zeros(n)
        d_star[i_star] = Q / c[i_star]
    elif alpha == 1:
        d_star = Q / (n * c)
    elif alpha == 'inf':
        sum_cost_per_utility = np.sum(c / utility)
        d_star = (1 / utility) * (Q / sum_cost_per_utility)
    else: # General alpha
        common_terms = np.power(c, -1/alpha) * np.power(utility, (1-alpha)/alpha)
        denominator = np.sum(c * common_terms)
        d_star = (Q * common_terms) / denominator

    # --- 2. Compute Gradient ∂d*/∂r ---
    if alpha == 0:
        # The argmax operation is non-differentiable.
        # The gradient is 0 almost everywhere. Finite difference is the practical approach.
        pass # grad_d_r is already zeros
    elif alpha == 1:
        # d* does not depend on r, so the gradient is zero.
        pass # grad_d_r is already zeros
    elif alpha == 'inf':
        for i in range(n):
            for k in range(n):
                if i == k: # Diagonal: ∂d_i*/∂r_i
                    grad_d_r[i, i] = -d_star[i] / r[i] * (1 - c[i] * d_star[i] / Q)
                else: # Off-diagonal: ∂d_i*/∂r_k
                    grad_d_r[i, k] = d_star[i] * d_star[k] * c[k] / (r[k] * Q)
    else: # General alpha
        # Precompute the common term from the derivative
        term = (1 - alpha) / (alpha * r) # Shape (n,)
        for i in range(n):
            for k in range(n):
                if i == k: # Diagonal: ∂d_i*/∂r_i
                    grad_d_r[i, i] = d_star[i] * term[i] * (1 - c[i] * d_star[i] / Q)
                else: # Off-diagonal: ∂d_i*/∂r_k
                    grad_d_r[i, k] = -d_star[i] * d_star[k] * c[k] * term[k] / Q

    return d_star, grad_d_r

In [17]:
def AlphaFairness(util, alpha):
    if isinstance(util, torch.Tensor):
        util = util.detach().cpu().numpy() if isinstance(util, torch.Tensor) else util
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha) / (1-alpha))


def solve_closed_form(g, r, c, alpha, Q):

    g = g.detach().cpu().numpy() if isinstance(g, torch.Tensor) else g
    r = r.detach().cpu().numpy() if isinstance(r, torch.Tensor) else r
    c = c.detach().cpu().numpy() if isinstance(c, torch.Tensor) else c
    if c.shape != r.shape or c.shape != g.shape:
        raise ValueError("c, r, and g must have the same shape.")
    if np.any(c <= 0):
        raise ValueError("All cost values must be positive.")
    if np.any(r <= 0):
        raise ValueError("All risk values must be positive.")
    if np.any(g <= 0):
        raise ValueError("All gain factors must be positive.")
    
    n = len(c)
    utility = r * g
    
    if alpha == 0:
        ratios = utility / c
        sorted_indices = np.argsort(-ratios)  # Descending order
        d_star_closed = np.zeros(n)
        d_star_closed[sorted_indices[0]] = Q / c[sorted_indices[0]]
        
    elif alpha == 1:
        d_star_closed = Q / (n * c)
    
    elif alpha == 'inf':
        d_star_closed = (Q * c) / (utility * np.sum(c * c / utility))
        
    else:
        if alpha <= 0:
            raise ValueError("Alpha must be positive for general case.")
        
        # This vector is common to the numerator of d_i and the terms in the sum
        # It corresponds to c_i^(-1/alpha) * utility_i^(1/alpha - 1)
        common_terms = np.power(c, -1/alpha) * np.power(utility, 1/alpha - 1)
        
        # The correct denominator is Σ_j(c_j * common_term_j)
        denominator = np.sum(c * common_terms)
        
        if denominator == 0:
            raise ValueError("Denominator is zero in closed-form solution.")
            
        # The numerator of d_i is Q * common_term_i
        d_star_closed = (Q * common_terms) / denominator
    
    # if not np.isclose(np.sum(c * d_star_closed), Q, rtol=1e-5):
    #     raise ValueError("Solution does not satisfy budget constraint.")
    obj = AlphaFairness(d_star_closed * utility, alpha)
        
    return d_star_closed, obj

def compute_gradient_closed_form(g, r, c, alpha, Q):
    """
    Compute the analytical gradient of the optimal solution with respect to r.

    This function computes the gradient matrix where each element (i, k) is the partial derivative
    of d_i* with respect to r_k.

    Parameters:
    - g (np.ndarray): Gain factors (g_i), shape (n,)
    - r (np.ndarray): Risk values (r_i), shape (n,)
    - c (np.ndarray): Cost values (c_i), shape (n,)
    - alpha (float or str): Fairness parameter. Can be 0, 1, 'inf', or a positive real number.
    - Q (float): Total budget.

    Returns:
    - gradient (np.ndarray): Gradient matrix of shape (n, n)
    """
    if alpha == 1:
        S = np.sum(c / (r * g))

    if alpha == 0:
        # Utilitarian case: Allocate everything to the individual with the highest ratio
        ratios = (r * g) / c
        i_star = np.argmax(ratios)
        # Gradient is Q * g_i / c_i at the allocated index, zero elsewhere
        gradient[i_star, i_star] = Q * g[i_star] / c[i_star]
        return gradient

    elif alpha == 'inf':
        # Maximin case
        n = len(c)
        utility = r * g  # Shape: (n,)
        S = np.sum(c**2 / utility)  # Scalar

        # Compute d_star
        d_star, _ = solve_closed_form(g,r,c, alpha='inf', Q=Q)  # Shape: (n,)

        # Initialize gradient matrix
        gradient = np.zeros((n, n))

        for i in range(n):
            for k in range(n):
                if i == k:
                    # ∂d_i*/∂r_i = -d_i*/r_i - (d_i* * c_i) / (r_i * g_i * S)
                    gradient[i, k] = -d_star[i] / r[i] - (d_star[i] * c[i]) / (r[i] * g[i] * S)
                else:
                    # ∂d_i*/∂r_k = (d_i* * c_k^2) / (c_i * r_k^2 * g_k * S)
                    gradient[i, k] = (d_star[i] * c[k]**2) / (c[i] * r[k]**2 * g[k] * S)
        return gradient

    else:
        # General alpha case
        if not isinstance(alpha, (int, float)):
            raise TypeError("Alpha must be a positive real number, 0, 1, or 'inf'.")
        if alpha <= 0:
            raise ValueError("Alpha must be positive for gradient computation.")

        # Compute the optimal decision variables
        d_star, _ = solve_closed_form(g, r, c, alpha, Q)  # Shape: (n,)

        # Compute the term (1/alpha - 1) * g / r
        term = (1.0 / alpha - 1.0) * g / r  # Shape: (n,)

        # Compute the outer product for off-diagonal elements
        # Each element (i, k) = -d_star[i] * d_star[k] * term[k] / Q
        gradient = -np.outer(d_star, d_star * term*c) / Q  # Shape: (n, n)

        # Compute the diagonal elements
        # Each diagonal element (i, i) = d_star[i] * term[i] * (1 - d_star[i]/Q)
        diag_elements = d_star * term * (1 - c*d_star / Q)  # Shape: (n,)

        # Set the diagonal elements
        np.fill_diagonal(gradient, diag_elements)

        return gradient

In [18]:
def get_finite_difference_gradient(g, r, c, alpha, Q, eps=1e-6):
    """
    Computes the gradient of d* w.r.t. r using finite differences for verification.
    """
    n = len(r)
    grad_d_r_fd = np.zeros((n, n))
    
    # Base case d*
    d_star_base, _ = solve_d_and_gradient_analytical(g, r, c, alpha, Q)

    for k in range(n):
        # Perturb r_k
        r_plus = r.copy()
        r_plus[k] += eps
        d_star_plus, _ = solve_d_and_gradient_analytical(g, r_plus, c, alpha, Q)
        
        # Compute the change
        grad_d_r_fd[:, k] = (d_star_plus - d_star_base) / eps
        
    return grad_d_r_fd

# --- Main Verification Script ---
if __name__ == '__main__':
    # Setup problem
    n_items = 50
    np.random.seed(42)
    g_test = np.ones(n_items)
    r_test = np.random.rand(n_items) + 0.1
    c_test = np.random.rand(n_items) + 0.1
    Q_test = 100.0
    
    print("--- Verifying Analytical Gradients ---\n")
    
    # Test for different alpha values
    alphas_to_test = [0.5, 2.0]

    for alpha_val in alphas_to_test:
        try:
            # 1. Get analytical solution
            _, grad_analytical = solve_d_and_gradient_analytical(g_test, r_test, c_test, alpha_val, Q_test)
            
            # 2. Get finite-difference solution
            grad_fd = get_finite_difference_gradient(g_test, r_test, c_test, alpha_val, Q_test)
            
            # 3. Compare the results
            is_close = np.allclose(grad_analytical, grad_fd, atol=1e-4)
            status = "✅ Matches" if is_close else "❌ Mismatch"
            
            print(f"Alpha = {str(alpha_val):<4} | Status: {status}")
            if not is_close:
                # Print the difference if they don't match
                print("Max difference:", np.max(np.abs(grad_analytical - grad_fd)))

        except Exception as e:
            print(f"Alpha = {str(alpha_val):<4} | Error: {e}")

    # Note on alpha = 0
    print("\nNote: alpha=0 is non-differentiable and its analytical gradient is zero almost everywhere.")
    print("Finite difference is the only practical method for alpha=0 if a gradient is needed.")

--- Verifying Analytical Gradients ---

Alpha = 0.5  | Status: ✅ Matches
Alpha = 2.0  | Status: ✅ Matches

Note: alpha=0 is non-differentiable and its analytical gradient is zero almost everywhere.
Finite difference is the only practical method for alpha=0 if a gradient is needed.


In [19]:
print("Analytical Gradient:\n", grad_analytical)
print("\nFinite Difference Gradient:\n", grad_fd)

Analytical Gradient:
 [[-2.54822573e+00  1.90985158e-02  2.95407406e-02 ...  3.27083560e-02
   1.49729316e-02  6.59437082e-02]
 [ 5.16835705e-02 -8.64201573e-01  2.19475534e-02 ...  2.43009611e-02
   1.11242714e-02  4.89934586e-02]
 [ 5.32917166e-02  1.46309172e-02 -1.12119446e+00 ...  2.50570911e-02
   1.14704056e-02  5.05179014e-02]
 ...
 [ 8.66530860e-02  2.37900787e-02  3.67974429e-02 ... -2.45479780e+00
   1.86510420e-02  8.21428231e-02]
 [ 1.74018351e-01  4.77756818e-02  7.38973143e-02 ...  8.18212276e-02
  -4.76767629e+00  1.64960757e-01]
 [ 2.03658255e-01  5.59131375e-02  8.64839714e-02 ...  9.57575356e-02
   4.38350075e-02 -1.25742405e+01]]

Finite Difference Gradient:
 [[-2.54822178e+00  1.90985023e-02  2.95407143e-02 ...  3.27083169e-02
   1.49729145e-02  6.59435360e-02]
 [ 5.16834902e-02 -8.64200962e-01  2.19475338e-02 ...  2.43009322e-02
   1.11242588e-02  4.89933309e-02]
 [ 5.32916340e-02  1.46309072e-02 -1.12119346e+00 ...  2.50570613e-02
   1.14703926e-02  5.05177697e-0

Running SLOW version...

Running FAST version...

Results from both versions match.
